In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
import plotly.graph_objects as go

In [69]:
poses = pd.read_csv('./poses/00.txt', header=None, sep=' ')
calib = pd.read_csv('./00_gray/calib.txt', header=None, sep=' ', index_col=0)

In [70]:
num_images = 1001

dir_images_0 = './00_gray/image_0/'
dir_images_1 = './00_gray/image_1/'

images_0_files = os.listdir(dir_images_0)[:num_images]
images_1_files = os.listdir(dir_images_1)[:num_images]

In [71]:
images_left = []
images_right = []

for file in images_0_files:
    img_path = os.path.join(dir_images_0, file)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    images_left.append(img)

for file in images_1_files:
    img_path = os.path.join(dir_images_1, file)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    images_right.append(img)

images_left = np.array(images_left)
images_right = np.array(images_right)

In [72]:
def compute_disparity(image_left, image_right, matcher='sgbm'):
    
    sad_window = 6
    num_disparities = sad_window * 16
    block_size = 7
    matcher_name = matcher
    
    if matcher_name == 'bm':
        matcher = cv2.StereoBM_create(numDisparities=num_disparities, blockSize=block_size)
    else:
        matcher = cv2.StereoSGBM_create(
                                        numDisparities=num_disparities, 
                                        blockSize=block_size,
                                        minDisparity=0,
                                        P1= 8 * 1 * block_size ** 2,
                                        P2= 33 * 1 * block_size ** 2,
                                        mode=cv2.StereoSGBM_MODE_SGBM_3WAY
                                      )
        
    disp_left = matcher.compute(image_left, image_right).astype(np.float32) / 16
    return disp_left

In [73]:
def decompose_proj(P):
    k, r, t, _, _, _, _ = cv2.decomposeProjectionMatrix(P)

    t = (t / t[3])[:3] # Bayad akharesh 1 bashe ta toye meter bashe
     
    return k, r, t

In [74]:
def calc_depth_Z(disp_l, k_left, t_left, t_right):
    b = t_right[0] - t_left[0]
    
    f = k_left[0][0]
    
    disp_l[disp_l==0.0] = 0.1
    disp_l[disp_l==-1.0] = 0.1
    
    depth_map = np.ones(disp_l.shape)
    ''' 
        f is focal length
        b is the base line
        
        d is the difference between x_L and x_R, 
            meaning the difference in horizontal pixel location of
            the point projected onto the left and right image planes.

        depth_map is Z    
    '''
    depth_map = (f * b) / disp_l 
    
    return depth_map

In [75]:
def stereo_2_depth(img_left, img_right, P0, P1, matcher='bm'):
    disp = compute_disparity(
                                img_left,
                                img_right,
                                matcher=matcher,
                            )
    k_left, r_left, t_left = decompose_proj(P0)
    k_right, r_right, t_right = decompose_proj(P1)
    
    depth = calc_depth_Z(disp, k_left, t_left, t_right)
    
    return depth

In [76]:
def extract_features(image, detector='sift', mask=None):
    if detector == 'sift':
        det = cv2.SIFT_create()
    elif detector == 'orb':
        det = cv2.ORB_create()
    elif detector == 'surf':
        det = cv2.xfeatures2d.SURF_create()
        
    kp, des = det.detectAndCompute(image, mask)
    
    return kp, des

In [77]:
def match_features(des1, des2, matching='BF', detector='sift', sort=True, k=2, dist_threshold=0.4):
    if matching == 'BF':
        if detector == 'sift':
            matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)
        elif detector == 'orb':
            matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING2, crossCheck=False)
            
        matches = matcher.knnMatch(des1, des2, k=k)
    
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=150)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        matches = matcher.knnMatch(des1, des2, k=k)
    
    if sort:
        matches = sorted(matches, key = lambda x:x[0].distance)

    filtered_match = []
    for m, n in matches:
        if m.distance <= dist_threshold*n.distance:
            filtered_match.append(m)

    return filtered_match

In [78]:
def estimate_motion(matches, kp1, kp2, k, depth, max_depth=3000, confidence=0.99):
    cx = k[0, 2]
    cy = k[1, 2]
    fx = k[0, 0]
    fy = k[1, 1]
    
    rmat = np.eye(3)
    tmat = np.zeros((3, 1))
    
    image_points1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    image_points2 = np.float32([kp2[m.trainIdx].pt for m in matches])
    
    ''' X Y Z '''
    object_points = np.zeros((0, 3))
    delete = []
    
    '''2d coordinates'''
    for i, (u, v) in enumerate(image_points1): 
        Z = depth[int(round(v)), int(round(u))]
        
        if Z > max_depth:
            delete.append(i)
            continue
        
        X = Z * (u - cx) / fx
        Y = Z * (v - cy) / fy
        
        object_points = np.vstack((
                                    object_points, 
                                    np.array([X, Y, Z])
                                   ))
        
    image_points1 = np.delete(image_points1, delete, 0)
    image_points2 = np.delete(image_points2, delete, 0)
    
    _, rvec, tvec, inliers = cv2.solvePnPRansac(object_points, image_points2, k, None, confidence=confidence)
    
    '''rvec 3x1 to rmat 3x3'''
    rmat = cv2.Rodrigues(rvec)[0]
    
    '''  [R|t] '''
    return rmat, tvec, object_points

In [79]:
def VO():
    start = datetime.datetime.now()
    print(f'Start Time = {start}')
    num_frames = num_images - 1
    
    P0 = np.array(calib.loc['P0:']).reshape(3, 4)
    P1 = np.array(calib.loc['P1:']).reshape(3, 4)
    
    T_tot = np.eye(4)
    trajectory = np.zeros((num_frames + 1, 3, 4))
    trajectory[0] = T_tot[:3, :]
    
    k_left, r_left, t_left = decompose_proj(P0)
    
    for i in tqdm(range(num_frames), desc="Processing Frames", unit="frame"):
        image_left = images_left[i]
        image_right = images_right[i]
        
        image_plus1 = images_left[i + 1]
        
        depth = stereo_2_depth(image_left, image_right, P0, P1, 'sgbm')
        
        kp0, des0 = extract_features(image_left)
        kp1, des1 = extract_features(image_plus1)
        
        matches = match_features(des0, des1, matching='FLANN', dist_threshold=0.9)
        
        rmat, tvec, object_points = estimate_motion(matches, kp0, kp1, k_left, depth)
        
        # In homogeneous coordinates
        Tmat = np.eye(4)
        Tmat[:3, :3] = rmat
        Tmat[:3, 3] = tvec.T
        
        T_tot = T_tot.dot(np.linalg.inv(Tmat))
        
        trajectory[i + 1, :, :] = T_tot[:3, :]
    
    end = datetime.datetime.now()
    print(f'Total Time = {end - start}')
    return trajectory, object_points

In [80]:
def calculate_error(gt, est, num_frames):
    squared_diffs = (gt.iloc[:num_frames, 3] - est[:, 0, 3])**2
    + (gt.iloc[:num_frames, 7] - est[:, 1, 3])**2
    + (gt.iloc[:num_frames, 11] - est[:, 2, 3])**2
    
    rmse = np.sqrt(np.mean(squared_diffs))
    
    return rmse

In [81]:
trajectory, object_points = VO()
rmse = calculate_error(poses, trajectory, num_images)

print('==========================')
print(f'RMSE: {rmse}')
print('==========================')

Start Time = 2024-11-20 11:50:02.552882


Processing Frames: 100%|██████████| 1000/1000 [04:15<00:00,  3.92frame/s]

Total Time = 0:04:15.126764
RMSE: 5.578838205142009


In [82]:
i = 0
print(object_points[i].round(4))
print([trajectory[i, 0, 3].round(4), trajectory[i, 1, 3].round(4), trajectory[i, 2, 3].round(4)])

[ -1.0676 -12.2571  77.229 ]
[0.0, 0.0, 0.0]


In [ ]:
def visualize_3D_points(points, trajectory, title="3D Points of Matched Features"):
    pose_xs = poses.iloc[:num_images, 3].values  
    pose_ys = poses.iloc[:num_images, 7].values  
    pose_zs = poses.iloc[:num_images, 11].values  

    traj_xs = trajectory[:, 0, 3]  
    traj_ys = trajectory[:, 1, 3]  
    traj_zs = trajectory[:, 2, 3]  
    
    feat_X = points[:, 0]
    feat_Y = points[:, 1]
    feat_Z = points[:, 2]

    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=pose_xs,
        y=pose_ys,
        z=pose_zs,
        mode='lines',
        name='Ground Truth',
        line=dict(color='chartreuse', width=5)
    ))

    fig.add_trace(go.Scatter3d(
        x=traj_xs,
        y=traj_ys,
        z=traj_zs,
        mode='lines',
        name='Calculated',
        line=dict(color='blue', width=5)
    ))
    
    fig.add_trace(go.Scatter3d(
        x=feat_X,
        y=feat_Y,
        z=feat_Z,
        mode='markers',
        name='Feature Points',
        marker=dict(size=2, color='red', opacity=0.6)
    ))

    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
        ),
        title='3D Trajectory Visualization',
        width=800,
        height=800,
    )

    fig.layout.template = 'plotly_dark'
    fig.show()

visualize_3D_points(object_points, trajectory)
